In [1]:
%matplotlib notebook
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm import tqdm_notebook as tqdm

textwidth = 6.
mpl.rcParams['axes.labelsize'] = 11.
mpl.rcParams['axes.titlesize'] = 11
mpl.rcParams['xtick.labelsize'] = 11
mpl.rcParams['ytick.labelsize'] = 11
mpl.rcParams['legend.fontsize'] = 11
mpl.rcParams['font.family'] = 'Open Sans'

In [4]:
N = 500

N_samples = 100

cf_w = .1
cf_w_in = 1.

sigm_w_e = .00001

eps_mu = 10.**-4.
eps_var = 10.**-3.

mu_y_target = -np.ones((N))*0.

#r_target = .9

T = 1000
T_skip_rec = 1
T_rec = int(T/T_skip_rec)

t_arr = np.arange(T_rec)*T_skip_rec

In [ ]:
W = np.random.normal(0.,1./(cf_w*N)**.5,(N,N)) * (np.random.rand(N,N) <= cf_w)
W[range(N),range(N)] = 0.

#W = W/np.max(np.abs(np.linalg.eigvals(W)))

a = np.ones((N))*.5
b = np.zeros((N))

w_in = np.random.normal(0.,sigm_w_e,(N,1)) * (np.random.rand(N,1) <= cf_w_in)

u_in = (np.random.rand(1,T) >= .5)*1.

y = np.ndarray((N))
X_r = np.ndarray((N))
X_e = np.ndarray((N))
#X_e = (w_in @ u_in).T
#X_e = np.random.normal(0.,1.,(T,N)) * w_in[:,0]
#X_e = np.random.normal(0.,.25,(T,N))

mu_y = np.ndarray((N))
mu_X_e = np.ndarray((N))
Var_y = np.ndarray((N))
Var_X_e = np.ndarray((N))

### recording
a_rec = np.ones((T_rec,N))
b_rec = np.ones((T_rec,N))

y_rec = np.ones((T_rec,N))
X_r_rec = np.ones((T_rec,N))
X_e_rec = np.ones((T_rec,N))

mu_y_rec = np.ones((T_rec,N))
mu_X_e_rec = np.ones((T_rec,N))
Var_y_rec = np.ones((T_rec,N))
Var_X_e_rec = np.ones((T_rec,N))
###

### first time step
X_e[:] = w_in @ u_in[:,0]
X_r[:] = np.zeros((N))
y[:] = np.tanh(X_r[:] + X_e[:])

mu_y[:] = y
mu_X_e[:] = X_e
Var_y[:] = .25
Var_X_e[:] = .25

#### Recording
a_rec[0,:] = a
b_rec[0,:] = b

y_rec[0,:] = y
X_r_rec[0,:] = X_r
X_e_rec[0,:] = X_e

mu_y_rec[0,:] = mu_y
mu_X_e_rec[0,:] = mu_X_e
Var_y_rec[0,:] = Var_y
Var_X_e_rec[0,:] = Var_X_e
####
###

for t in tqdm(range(1,T)):
    
    X_r[:] = a[:] * (W @ y[:])
    X_e[:] = w_in @ u_in[:,t]
    
    y[:] = np.tanh(X_r + X_e - b)
        
    mu_y[:] = (1.-eps_mu)*mu_y + eps_mu * y
    mu_X_e[:] = (1.-eps_mu)*mu_X_e + eps_mu * X_e
    
    Var_y[:] = (1.-eps_var)*Var_y + eps_var * (y - mu_y)**2.
    Var_X_e[:] = (1.-eps_var)*Var_X_e + eps_var * (X_e - mu_X_e)**2.
    
    y_squ_targ = 1.-1./(1.+2.*Var_y.mean() + 2.*Var_X_e)**.5
    
    a = a + eps_a * a * ((y**2.).mean() - (X_r**2.).mean())
    b = b + eps_b * (y - mu_y_target)
    
    a = np.maximum(0.1,a)    
    
    if t%T_skip_rec == 0:
        t_rec = int(t/T_skip_rec)
        
        #### Recording
        a_rec[t_rec,:] = a
        b_rec[t_rec,:] = b

        y_rec[t_rec,:] = y
        X_r_rec[t_rec,:] = X_r
        X_e_rec[t_rec,:] = X_e

        mu_y_rec[t_rec,:] = mu_y
        mu_X_e_rec[t_rec,:] = mu_X_e
        Var_y_rec[t_rec,:] = Var_y
        Var_X_e_rec[t_rec,:] = Var_X_e
        ####